1) Install Dependencies

In [ ]:
!pip install SimpleITK
!pip install batchgenerators
!pip install connected-components-3d
!pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 52.8 MB 196 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 61 kB 333 kB/s 
     |████████████████████████████████| 96 kB 3.8 MB/s 
  Created wheel for batchgenerators: filename=batchgenerators-0.24-py3-none-any.whl size=89210 sha256=0b2679a97744ce3871f9fa3db42c9d710ab46154df492fde309fbf69a1abc4ec
  Stored in directory: /root/.cache/pip/wheels/56/d7/ca/6b43d8fbc8e1d38a89e902cd0cf5aeeb35501ba3c95e3ce8e7
Successfully built batchgenerators


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.7 MB 5.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive')

#import relevant dependencies
import glob, os, functools
import numpy as np
import pandas as pd
import SimpleITK as sitk

Mounted at /content/drive


2) Define Resampling Functions


In [ ]:
# SOURCE: https://github.com/iantsen/hecktor/blob/main/src/data/utils.py
def resample_sitk_image(sitk_image,
                        new_spacing=[1, 1, 1],
                        new_size=None,
                        attributes=None,
                        interpolator=sitk.sitkLinear,
                        fill_value=0):
    """
    Resample a SimpleITK Image.
    Parameters
    ----------
    sitk_image : sitk.Image
        An input image.
    new_spacing : list of int
        A distance between adjacent voxels in each dimension given in physical units (mm) for the output image.
    new_size : list of int or None
        A number of pixels per dimension of the output image. If None, `new_size` is computed based on the original
        input size, original spacing and new spacing.
    attributes : dict or None
        The desired output image's spatial domain (its meta-data). If None, the original image's meta-data is used.
    interpolator
        Available interpolators:
            - sitk.sitkNearestNeighbor : nearest
            - sitk.sitkLinear : linear
            - sitk.sitkGaussian : gaussian
            - sitk.sitkLabelGaussian : label_gaussian
            - sitk.sitkBSpline : bspline
            - sitk.sitkHammingWindowedSinc : hamming_sinc
            - sitk.sitkCosineWindowedSinc : cosine_windowed_sinc
            - sitk.sitkWelchWindowedSinc : welch_windowed_sinc
            - sitk.sitkLanczosWindowedSinc : lanczos_windowed_sinc
    fill_value : int or float
        A value used for padding, if the output image size is less than `new_size`.
    Returns
    -------
    sitk.Image
        The resampled image.
    Notes
    -----
    This implementation is based on https://github.com/deepmedic/SimpleITK-examples/blob/master/examples/resample_isotropically.py
    """
    sitk_interpolator = interpolator

    # provided attributes:
    if attributes:
        orig_pixelid = attributes['orig_pixelid']
        orig_origin = attributes['orig_origin']
        orig_direction = attributes['orig_direction']
        orig_spacing = attributes['orig_spacing']
        orig_size = attributes['orig_size']

    else:
        # use original attributes:
        orig_pixelid = sitk_image.GetPixelIDValue()
        orig_origin = sitk_image.GetOrigin()
        orig_direction = sitk_image.GetDirection()
        orig_spacing = np.array(sitk_image.GetSpacing())
        orig_size = np.array(sitk_image.GetSize(), dtype=int)

    # new image size:
    if not new_size:
        new_size = orig_size * (orig_spacing / new_spacing)
        new_size = np.ceil(new_size).astype(int)  # Image dimensions are in integers
        new_size = [int(s) for s in new_size]  # SimpleITK expects lists, not ndarrays

    resample_filter = sitk.ResampleImageFilter()
    resample_filter.SetOutputSpacing(new_spacing)
    resample_filter.SetSize(new_size)
    resample_filter.SetOutputDirection(orig_direction)
    resample_filter.SetOutputOrigin(orig_origin)
    resample_filter.SetTransform(sitk.Transform())
    resample_filter.SetDefaultPixelValue(orig_pixelid)
    resample_filter.SetInterpolator(sitk_interpolator)
    resample_filter.SetDefaultPixelValue(fill_value)

    resampled_sitk_image = resample_filter.Execute(sitk_image)

    return resampled_sitk_image

In [ ]:
#Source: https://github.com/iantsen/hecktor/blob/main/src/data/utils.py
def get_attributes(sitk_image):
    """Get physical space attributes (meta-data) of the image."""
    attributes = {}
    attributes['orig_pixelid'] = sitk_image.GetPixelIDValue()
    attributes['orig_origin'] = sitk_image.GetOrigin()
    attributes['orig_direction'] = sitk_image.GetDirection()
    attributes['orig_spacing'] = np.array(sitk_image.GetSpacing())
    attributes['orig_size'] = np.array(sitk_image.GetSize(), dtype=np.int)
    return attributes


3) Define Rigid Registration Functions

In [ ]:
def nrrd_reg_rigid_ref(ref_path, moving_img, pt, mask):
    
    fixed_img = sitk.ReadImage(ref_path)
    #fixed_img = ref_path

    
    transform = sitk.CenteredTransformInitializer(
        fixed_img, 
        moving_img, 
        sitk.Euler3DTransform(), 
        sitk.CenteredTransformInitializerFilter.GEOMETRY
        )
    
    # multi-resolution rigid registration using Mutual Information
    registration_method = sitk.ImageRegistrationMethod()
    registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
    registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
    registration_method.SetMetricSamplingPercentage(0.01)
    registration_method.SetInterpolator(sitk.sitkLinear)

    registration_method.SetOptimizerAsGradientDescent(
        learningRate=1.0, 
        numberOfIterations=100, 
        convergenceMinimumValue=1e-6, 
        convergenceWindowSize=10
        )
    
    registration_method.SetOptimizerScalesFromPhysicalShift()
    registration_method.SetShrinkFactorsPerLevel(shrinkFactors=[4, 2, 1])
    registration_method.SetSmoothingSigmasPerLevel(smoothingSigmas=[2, 1, 0])
    registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()
    registration_method.SetInitialTransform(transform)
    final_transform = registration_method.Execute(fixed_img, moving_img) 
    print(final_transform)
    #do transformation
    ct_reg = sitk.Resample(moving_img, fixed_img, final_transform, sitk.sitkLinear, 0.0, moving_img.GetPixelID())
    pt_reg = sitk.Resample(pt, fixed_img, final_transform, sitk.sitkLinear, 0.0, pt.GetPixelID())
    #mask_reg = sitk.Resample(mask, fixed_img, final_transform, sitk.sitkNearestNeighbor, 0.0, mask.GetPixelID())
    #return ct_reg, pt_reg, mask_reg, final_transform
    return ct_reg, pt_reg, 0, final_transform

4)Define Crop Functions

In [ ]:
from SimpleITK.extra import GetArrayFromImage
from scipy import ndimage
import cv2
import matplotlib as plt
def crop_top(img, pet, label, crop_shape):
  # get image, arr, and spacing
  image_arr = GetArrayFromImage(img)
  image_spacing = img.GetSpacing()
  image_origin = img.GetOrigin()
  label_arr = GetArrayFromImage(label)
  label_spacing = label.GetSpacing()
  label_origin = label.GetOrigin()
  pet_arr = GetArrayFromImage(pet)
  pet_spacing = pet.GetSpacing()
  pet_origin = pet.GetOrigin()
  
  c,y,x = image_arr.shape
  ## Get center of mass to center the crop in Y plane
  mask_arr = np.copy(image_arr)
  mask_arr[mask_arr > -500] = 1
  mask_arr[mask_arr <= -500] = 0
  print("mask_arr min and max:", np.amin(mask_arr), np.amax(mask_arr))
  mask_arr = np.array(mask_arr)

  # find center of just connected components
  labels_in = np.array(mask_arr[76,:,:], dtype = np.uint8 )
  image = np.array(image_arr[76,:,:], dtype = np.uint8 )
  image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

  # find contours in the binary image
  contours, hierarchy = cv2.findContours(labels_in,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
  biggest = 0
  index = -1
  for j,ct in enumerate(contours):
    circular_area = cv2.contourArea(ct)*np.power(4*np.pi*cv2.contourArea(ct)/(np.power(cv2.arcLength(ct,True),2)),2)
    if circular_area > biggest:
      biggest = circular_area
      index = j
  image_copy = image.copy()
  contour = contours[index]

  M = cv2.moments(contour)

  # calculate x,y coordinate of center
  cX = int(M["m10"] / M["m00"])

  cY = int(M["m01"] / M["m00"])

  centermass = np.array([cX,cY])

  print("center of mass: ", centermass)
  startx = int(centermass[0] - crop_shape[0]//2)
  starty = int(centermass[1] - crop_shape[1]//2)
  startz = int(c - crop_shape[2] - 20 )
  print("start X,Y,Z: ", startx, starty, startz)
  if startx < 0:
    startx = 0
  if starty < 0:
    starty = 0
  if (starty+crop_shape[1] > y):
    print("yes")
    starty = starty-((starty+crop_shape[1])-y)
  if startz < 0:
    image_arr = np.pad(image_arr,
                ((0, 0),
                (abs(startx)//2, abs(startx)//2),
                (0, 0)), 'constant', constant_values=-1024)
    label_arr = np.pad(label_arr,
                ((0, 0),
                (abs(startx)//2, abs(startx)//2),
                (0, 0)), 'constant', constant_values=0)
    pet_arr = np.pad(pet_arr,
                ((0, 0),
                (abs(startx)//2, abs(startx)//2),
                (0, 0)), 'constant', constant_values=0)
    image_arr_crop = image_arr[startz:startz+crop_shape[2],starty:starty+crop_shape[1],startx:startx+crop_shape[0]]
    label_arr_crop = label_arr[startz:startz+crop_shape[2],starty:starty+crop_shape[1],startx:startx+crop_shape[0]]
    pet_arr_crop = pet_arr[startz:startz+crop_shape[2],starty:starty+crop_shape[1],startx:startx+crop_shape[0]]
  else:
    image_arr_crop = image_arr[startz:startz+crop_shape[2],starty:starty+crop_shape[1],startx:startx+crop_shape[0]]
    label_arr_crop = label_arr[startz:startz+crop_shape[2],starty:starty+crop_shape[1],startx:startx+crop_shape[0]]
    pet_arr_crop = pet_arr[startz:startz+crop_shape[2],starty:starty+crop_shape[1],startx:startx+crop_shape[0]]


  sitk_label = sitk.GetImageFromArray(label_arr_crop)
  image = sitk.GetImageFromArray(image_arr_crop)
  image.SetOrigin(img.GetOrigin())
  image.SetSpacing(img.GetSpacing())
  sitk_label.SetOrigin(label.GetOrigin())
  sitk_label.SetSpacing(label.GetSpacing())
  new_pet = sitk.GetImageFromArray(pet_arr_crop)
  new_pet.SetOrigin(pet.GetOrigin())
  new_pet.SetSpacing(pet.GetSpacing())
  text = str(c)+" "+str(y)+" "+str(x)+" "+str(startz)+" "+str(starty)+" "+str(startx)
  # save nrrd
  return image, new_pet, sitk_label,text

In [ ]:
def quick_crop(img, length):
  img_arr = sitk.GetArrayFromImage(img)
  img_arr = img_arr[length:img.GetSize()[2],:,:]
  new_img = sitk.GetImageFromArray(img_arr)
  new_img.SetSpacing(img.GetSpacing())
  new_img.SetOrigin(img.GetOrigin())
  return new_img

4) Main

In [ ]:
#filepaths

in_data = '/content/drive/MyDrive/DATA/hecktor2022-data/hecktor2022_testing/imagesTs'
in_label =  '/content/drive/MyDrive/DATA/hecktor2022-data/labelsTr'
ref_folder = '/content/drive/MyDrive/DATA/nnUNet_raw_data_base/nnUNet_raw_data/Task507_160x160x64/competition_data/'
names_path = '/content/drive/MyDrive/DATA/hecktor2022-data/hecktor2022_testing/hecktor2022_clinical_info_testing 2.csv'
out_img = '/content/drive/MyDrive/DATA/nnUNet_raw_data_base/nnUNet_raw_data/Task507_160x160x64/competition_data/imagesTs'
out_temp = '/content/drive/MyDrive/DATA/nnUNet_raw_data_base/nnUNet_raw_data/Task507_160x160x64/competition_data/temp_folder/'
out_label = '/content/drive/MyDrive/DATA/nnUNet_raw_data_base/nnUNet_raw_data/Task507_160x160x64/labelsTr/'
out_trans = '/content/drive/MyDrive/DATA/nnUNet_raw_data_base/nnUNet_raw_data/Task507_160x160x64/competition_data/transforms'

#patient names
df = pd.read_csv(names_path)
names = df['PatientID'].to_numpy()

code = 'HEK_'
#loop through patients
for i, name in enumerate(names):

  #grab files
  img_ct = sitk.ReadImage(os.path.join(in_data,name+'__CT.nii.gz'))
  img_pt = sitk.ReadImage(os.path.join(in_data,name+'__PT.nii.gz'))
  mask = sitk.ReadImage(os.path.join(in_label,name+'.nii.gz'))

  #resample to 1x1x3
  img_ct = resample_sitk_image(img_ct, new_spacing=[1, 1, 3],new_size= None, interpolator=sitk.sitkLinear)
  img_pt = resample_sitk_image(img_pt, new_spacing=[1, 1, 3],new_size= None, interpolator=sitk.sitkLinear)
  mask = resample_sitk_image(mask, new_spacing=[1, 1, 3],new_size= None, interpolator=sitk.sitkNearestNeighbor)

  #crop for uniform sizing
  size_ct = img_ct.GetSize()[2]
  size_pt = img_pt.GetSize()[2]
  size_mask = mask.GetSize()[2]

  #check for uncropped images
  if(200<size_ct):
    img_ct = quick_crop(img_ct, int(size_ct*.65))
    img_pt = quick_crop(img_pt, int(size_pt*.65))
    mask = quick_crop(mask,int(size_mask*.65))

#save reference
  if i == 0:
    sitk.WriteImage(img_ct,os.path.join(out_temp,code+str(i).zfill(3)+'_0000.nii.gz'))
    sitk.WriteImage(img_pt, os.path.join(out_temp,code+str(i).zfill(3)+'_0001.nii.gz'))
    sitk.WriteImage(mask, os.path.join(out_temp,code+str(i).zfill(3)+'.nii.gz'))
    cr_img_ct, cr_img_pt, cr_mask = img_ct,img_pt,mask
  else:
    # Registration
    mask = 0
    cr_mask = 0
    cr_img_ct, cr_img_pt, cr_mask, transform = nrrd_reg_rigid_ref(os.path.join(out_temp,code+'000_'+'0000.nii.gz'), img_ct, img_pt, mask)
    # save tranformation
    sitk.WriteTransform(transform, os.path.join(out_trans,code+str(i).zfill(3)+'.txt'))

    #toggle crop
    cr_img_ct, cr_img_pt, cr_mask, text = crop_top(cr_img_ct, cr_img_pt, cr_mask, [160,160,72])
    
    #save crop parameters
    with open(os.path.join(out_trans,code+str(i).zfill(3)+'crop.txt'), 'w') as f:
      f.write(text)

    #save files
    sitk.WriteImage(cr_img_ct, os.path.join(out_img,code+str(i).zfill(3)+'_0000.nii.gz'))
    sitk.WriteImage(cr_img_pt, os.path.join(out_img,code+str(i).zfill(3)+'_0001.nii.gz'))
    sitk.WriteImage(cr_mask, os.path.join(out_label,code+str(i).zfill(3)+'.nii.gz'))

# crop reference file
#grab files
name = 'MDA-202'
cr_img_ct = sitk.ReadImage(os.path.join(in_data,name+'__CT.nii.gz'))
cr_img_pt = sitk.ReadImage(os.path.join(in_data,name+'__PT.nii.gz'))
cr_mask = sitk.ReadImage(os.path.join(out_label,code+'000.nii.gz'))
cr_mask = 0
cr_img_ct, cr_img_pt, cr_mask, transform = nrrd_reg_rigid_ref(os.path.join(out_temp,code+'000'+'_0000.nii.gz'), cr_img_ct, cr_img_pt, cr_mask)
# save tranformation
sitk.WriteTransform(transform, os.path.join(out_trans,code+'000.txt'))

#toggle crop
cr_img_ct, cr_img_pt, cr_mask, text = crop_top(cr_img_ct, cr_img_pt, cr_mask, [160,160,72])
with open(os.path.join(out_trans,code+'000crop.txt'), 'w') as f:
      f.write(text)
#save image
sitk.WriteImage(cr_img_ct,os.path.join(out_img,code+'000'+'_0000.nii.gz') )
sitk.WriteImage(cr_img_pt, os.path.join(out_img,code+'000'+'_0001.nii.gz'))
sitk.WriteImage(cr_mask, os.path.join(out_label,code+'000'+'.nii.gz'))

104
MDA-306
itk::simple::Euler3DTransform
 Euler3DTransform (0x2cd78e0)
   RTTI typeinfo:   itk::Euler3DTransform<double>
   Reference Count: 3
   Modified Time: 11154235
   Debug: Off
   Object Name: 
   Observers: 
     none
   Matrix: 
     0.999999 0.00134815 -3.05409e-06 
     -0.00134816 0.999992 -0.0037909 
     -2.05666e-06 0.0037909 0.999993 
   Offset: [1.78484, -0.383763, -800.056]
   Center: [-0.5, -0.5, 35.82]
   Translation: [1.78406, -0.518875, -800.059]
   Inverse: 
     0.999999 -0.00134816 -2.05666e-06 
     0.00134815 0.999992 0.0037909 
     -3.05409e-06 -0.0037909 0.999993 
   Singular: 0
   Euler's angles: AngleX=0.00379091 AngleY=2.05668e-06 AngleZ=-0.00134816
   m_ComputeZYX = 0

mask_arr min and max: 0.0 1.0
center of mass:  [ 57 272]
start X,Y,Z:  -23 192 56
(160, 160, 72)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: RuntimeWarning: invalid value encountered in double_scalars


106
MDA-308
itk::simple::Euler3DTransform
 Euler3DTransform (0x2cd5380)
   RTTI typeinfo:   itk::Euler3DTransform<double>
   Reference Count: 3
   Modified Time: 12004503
   Debug: Off
   Object Name: 
   Observers: 
     none
   Matrix: 
     0.999982 -0.00603736 -0.000679217 
     0.006043 0.999945 0.00862839 
     0.000627086 -0.00863234 0.999963 
   Offset: [2.33412, -2.14895, -926.481]
   Center: [-0.5, -0.5, 35.82]
   Translation: [2.31282, -1.84287, -926.478]
   Inverse: 
     0.999982 0.006043 0.000627086 
     -0.00603736 0.999945 -0.00863234 
     -0.000679217 0.00862839 0.999963 
   Singular: 0
   Euler's angles: AngleX=-0.00863245 AngleY=-0.00062711 AngleZ=0.00603762
   m_ComputeZYX = 0

mask_arr min and max: 0.0 1.0
center of mass:  [247 234]
start X,Y,Z:  167 154 56
(160, 160, 72)
150
MDA-352
itk::simple::Euler3DTransform
 Euler3DTransform (0x2cd6220)
   RTTI typeinfo:   itk::Euler3DTransform<double>
   Reference Count: 3
   Modified Time: 12852307
   Debug: Off
   Object

6) JSON generation

In [ ]:
#    Copyright 2020 Division of Medical Image Computing, German Cancer Research Center (DKFZ), Heidelberg, Germany
#
#    Licensed under the Apache License, Version 2.0 (the "License");
#    you may not use this file except in compliance with the License.
#    You may obtain a copy of the License at
#
#        http://www.apache.org/licenses/LICENSE-2.0
#
#    Unless required by applicable law or agreed to in writing, software
#    distributed under the License is distributed on an "AS IS" BASIS,
#    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#    See the License for the specific language governing permissions and
#    limitations under the License.


from typing import Tuple
import numpy as np
from batchgenerators.utilities.file_and_folder_operations import *


def get_identifiers_from_splitted_files(folder: str):
    uniques = np.unique([i[:-9] for i in subfiles(folder, suffix='.nii.gz', join=False)])
    return uniques


def generate_dataset_json(output_file: str, imagesTr_dir: str, imagesTs_dir: str, modalities: Tuple,
                          labels: dict, dataset_name: str, sort_keys=True, license: str = "hands off!", dataset_description: str = "",
                          dataset_reference="", dataset_release='0.0'):
    """
    :param output_file: This needs to be the full path to the dataset.json you intend to write, so
    output_file='DATASET_PATH/dataset.json' where the folder DATASET_PATH points to is the one with the
    imagesTr and labelsTr subfolders
    :param imagesTr_dir: path to the imagesTr folder of that dataset
    :param imagesTs_dir: path to the imagesTs folder of that dataset. Can be None
    :param modalities: tuple of strings with modality names. must be in the same order as the images (first entry
    corresponds to _0000.nii.gz, etc). Example: ('T1', 'T2', 'FLAIR').
    :param labels: dict with int->str (key->value) mapping the label IDs to label names. Note that 0 is always
    supposed to be background! Example: {0: 'background', 1: 'edema', 2: 'enhancing tumor'}
    :param dataset_name: The name of the dataset. Can be anything you want
    :param sort_keys: In order to sort or not, the keys in dataset.json
    :param license:
    :param dataset_description:
    :param dataset_reference: website of the dataset, if available
    :param dataset_release:
    :return:
    """
    train_identifiers = get_identifiers_from_splitted_files(imagesTr_dir)

    if imagesTs_dir is not None:
        test_identifiers = get_identifiers_from_splitted_files(imagesTs_dir)
    else:
        test_identifiers = []

    json_dict = {}
    json_dict['name'] = dataset_name
    json_dict['description'] = dataset_description
    json_dict['tensorImageSize'] = "4D"
    json_dict['reference'] = dataset_reference
    json_dict['licence'] = license
    json_dict['release'] = dataset_release
    json_dict['modality'] = {str(i): modalities[i] for i in range(len(modalities))}
    json_dict['labels'] = {str(i): labels[i] for i in labels.keys()}

    json_dict['numTraining'] = len(train_identifiers)
    json_dict['numTest'] = len(test_identifiers)
    json_dict['training'] = [
        {'image': "./imagesTr/%s.nii.gz" % i, "label": "./labelsTr/%s.nii.gz" % i} for i
        in
        train_identifiers]
    json_dict['test'] = ["./imagesTs/%s.nii.gz" % i for i in test_identifiers]

    if not output_file.endswith("dataset.json"):
        print("WARNING: output file name is not dataset.json! This may be intentional or not. You decide. "
              "Proceeding anyways...")
    save_json(json_dict, os.path.join(output_file), sort_keys=sort_keys)

In [ ]:
#filepaths
json_path = '/content/drive/MyDrive/DATA/nnUNet_raw_data_base/nnUNet_raw_data/Task503_node_CT/'
print(json_path)
#generate json
out_label = '/content/drive/MyDrive/DATA/nnUNet_raw_data_base/nnUNet_raw_data/Task503_node_CT/labelsnTr/'
out_test = '/content/drive/MyDrive/DATA/nnUNet_raw_data_base/nnUNet_raw_data/Task503_node_CT/imagesTs/'
#format is path to data, path to labels, path to testing data, modalities, encoding, dataset name
generate_dataset_json(os.path.join(json_path,'dataset.json'), out_label, out_test, ('CT'),{0: 'background', 1: 'node'},dataset_name = 'HECKTOR2022')

/content/drive/MyDrive/DATA/nnUNet_raw_data_base/nnUNet_raw_data/Task503_node_CT/


NameError: ignored

7) Invert Registration

In [ ]:
def invert_registration(label, CT, transform):
  inverse_transform = transform.GetInverse()
  #return transformed label
  return sitk.Resample(label, CT, inverse_transform, sitk.sitkNearestNeighbor,0.0, label.GetPixelID())

In [ ]:
#set path to transformation files
out_trans = '/content/drive/MyDrive/DATA/nnUNet_raw_data_base/nnUNet_raw_data/Task509_ProcessedCompetitionData/transforms/'
#set path to original data file
in_data = '/content/drive/MyDrive/DATA/hecktor2022-data/hecktor2022_testing/imagesTs/'
#set path to label file
label_files = '/content/drive/MyDrive/DATA/RESULTS_FOLDER_MNETNNUNET2/nnUNet/3d_fullres/Task508_hecktorfinal/myTrainer__nnUNetPlansv2.1/finalall/'
#names path
names_path = '/content/drive/MyDrive/DATA/hecktor2022-data/hecktor2022_testing/hecktor2022_clinical_info_testing 2.csv'
#set path to fixed_labels
out_fixed = '/content/drive/MyDrive/DATA/nnUNet_raw_data_base/nnUNet_raw_data/Task508_hecktorfinal/mnet+nnunet_aligned_labels/'
#processed images
in_img_path = '/content/drive/MyDrive/DATA/nnUNet_raw_data_base/nnUNet_raw_data/Task508_hecktorfinal/imagesTs'
#patient names
df = pd.read_csv(names_path)
names = df['PatientID'].to_numpy()

for i, name in enumerate(names):
  name = 'HEK_'+str(i).zfill(3)
  ct_name = names[i]
  print(name)
  print(ct_name)

  #grab files

  #original CT
  ct = sitk.ReadImage(os.path.join(in_data,ct_name+'__CT.nii.gz'))
  e_ct = ct
  #processed CT
  p_ct = sitk.ReadImage(os.path.join(in_img_path, name+'_0000.nii.gz'))

  #label
  #get reference file spacing and origin
  #label_arr = np.load(os.path.join(label_files,name+'.npy'))
  label = sitk.ReadImage(os.path.join(label_files,name+'.nii.gz'))
  label_arr = GetArrayFromImage(label)
  print(label_arr.shape)

  #undo cropping
  with open(os.path.join(out_trans, name+ 'crop.txt')) as f:
    all_lines = [[int(num) for num in line.split()] for line in f]
    all_lines = all_lines[0]
  print(all_lines)

  #un-crop for uniform sizing
  size_ct = ct.GetSize()[2]
  #check for uncropped images
  if(200<size_ct):
    ct = quick_crop(ct, int(size_ct*.65))

  undo = np.zeros(shape=(all_lines[0],all_lines[1],all_lines[2]))
  #crop length
  crop_shape = [172,172,76]
  startz = all_lines[3]
  starty = all_lines[4]
  startx = all_lines[5]


  undo[startz:startz+crop_shape[2],starty:starty+crop_shape[1],startx:startx+crop_shape[0]] = label_arr


  undo_label = sitk.GetImageFromArray(undo)
  undo_label.SetOrigin(p_ct.GetOrigin())
  undo_label.SetSpacing(p_ct.GetSpacing())

  #transformation
  read_transform = sitk.ReadTransform(os.path.join(out_trans,name+'.txt'))

  #invert
  inverted_label = invert_registration(undo_label, ct, read_transform)

  origin = inverted_label.GetOrigin()
  spacing = inverted_label.GetSpacing()

  if(200<size_ct):
    invert_arr = GetArrayFromImage(inverted_label)
    invert_arr = np.pad(invert_arr, (((e_ct.GetSize()[2]-ct.GetSize()[2]),0),(0,0),(0,0)), 'constant',constant_values = 0)
    print(invert_arr.shape)
    inverted_label = sitk.GetImageFromArray(invert_arr)
    inverted_label.SetOrigin(origin)
    inverted_label.SetSpacing(spacing)

  #save transformed label
  sitk.WriteImage(inverted_label, os.path.join(out_fixed,ct_name+'.nii.gz'))